In [1]:
import pandas as pd
import numpy as np
import sklearn

# EDA

In [2]:
cells = pd.read_csv("TG_LASSO_GeneExpressionInput/TG_LASSO_GeneExpressionInput/gdsc_expr_postCB.csv",  index_col=0)

Columns are cell lines are samples, so we flip it

In [3]:
cellsT = cells.T

In [4]:
cellsT.describe()

,ENSG00000000003,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,ENSG00000001460,...,ENSG00000261685,ENSG00000261701,ENSG00000261857,ENSG00000263002,ENSG00000263843,ENSG00000264247,ENSG00000264575,ENSG00000265298,ENSG00000265929,ENSG00000266282
count,979.000000,979.000000,979.000000,979.000000,979.000000,979.000000,979.000000,979.000000,979.000000,979.000000,...,979.000000,979.000000,979.000000,979.000000,979.000000,979.000000,979.000000,979.000000,979.000000,979.000000
mean,5.616081,8.184602,3.240549,2.625700,2.826448,3.688519,6.152474,4.771073,4.560786,2.441220,...,2.396055,1.299202,2.471192,3.045121,2.317518,2.848518,3.307870,6.373915,1.679950,5.542397
std,1.615159,0.578248,0.558842,0.617289,1.086662,1.719458,1.469351,1.095915,0.664583,0.586341,...,1.626232,1.394185,1.901716,0.487297,0.516274,0.641210,1.051554,0.730267,1.477778,0.790017
min,2.478080,1.010429,2.060476,1.423062,2.132002,1.997728,2.353709,1.845859,3.075438,1.146083,...,1.157664,-0.556403,0.965709,1.792216,0.972288,1.906680,1.164410,4.249395,0.357473,1.719171
25%,4.988018,7.822825,2.857939,2.178742,2.449539,2.493742,5.848949,4.069281,4.100525,2.033299,...,1.638643,0.782973,1.646073,2.713212,1.967198,2.460341,2.529131,5.851927,1.045082,5.043732
50%,6.137223,8.177196,3.145284,2.509144,2.564273,2.747983,6.605780,4.543546,4.484364,2.348046,...,1.847427,1.184600,1.862778,3.019886,2.269962,2.681192,3.214411,6.350442,1.287334,5.544246
75%,6.704101,8.544412,3.497180,2.980124,2.695955,4.634769,7.097506,5.284285,4.928840,2.759277,...,2.264629,1.558609,2.240749,3.361101,2.628525,3.003758,3.987572,6.863172,1.604504,5.977486
max,9.071790,10.089456,5.876035,5.036346,10.795663,9.950181,8.890287,9.282193,7.836351,5.607175,...,13.124424,26.905691,11.381738,4.731316,4.489086,6.825467,6.493159,8.911424,9.573914,9.177348


# 1 Dimensionality reduction

## a)

Let’s start by visualizing the data using dimensionality reduction methods. Use PCA, 
UMAP and t-SNE algorithms to map the samples onto a 2-D space for visualization 
(you do not need to implement these methods from scratch. You only need to find 
their implementation). 

PCA

In [5]:
from sklearn.preprocessing import StandardScaler

x = cellsT.values
x.shape

(979, 13941)

In [6]:
# Standardizing the features
x = StandardScaler().fit_transform(x)

In [7]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
drPCA = pca.fit_transform(x)

In [8]:
drPCA.shape

(979, 2)

UMAP

In [9]:
import umap

In [10]:
reducer = umap.UMAP()
drUMAP = reducer.fit_transform(x)
drUMAP.shape

(979, 2)

### t-SNE

In [11]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, learning_rate='auto', init='random')
drTSNE = tsne.fit_transform(x)
drTSNE.shape

(979, 2)

### Plot

In [12]:
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
sns.scatterplot(data=drPCA, x=0, y=1)
plt.show()

ValueError: If using all scalar values, you must pass an index